In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import os
import folium

In [6]:
expl_df=pd.read_csv('data/cleaned_data.csv')

In [31]:
data = expl_df[['latitude', 'longitude']].head(10).values.tolist()


# Створюємо карту з центром на середніх координатах і збільшеним масштабом
mymap = folium.Map(location=[expl_df['latitude'].mean(), expl_df['longitude'].mean()], zoom_start=12)


# Додаємо маркери на карту за координатами
for index, row in expl_df.head(10).iterrows():
    # Отримуємо значення sub_event_type для поточного рядка
    sub_event_type = row['sub_event_type']
    
    # Створюємо текст для popup, що включає значення sub_event_type
    popup_text = 'Sub-event type: {}'.format(sub_event_type)
    
    # Додаємо маркер з popup_text
    folium.Marker([row['latitude'], row['longitude']], popup=popup_text).add_to(mymap)

# Відображаємо карту
mymap.save('map.html')

In [25]:

# Групуємо дані за розташуванням і обчислюємо загальну кількість смертей для кожного розташування
location_fatalities = expl_df[expl_df['fatalities'] > 0].groupby(['latitude', 'longitude', 'location'])['fatalities'].sum().reset_index()

# Створюємо карту
mymap = folium.Map(location=[location_fatalities['latitude'].mean(), location_fatalities['longitude'].mean()], zoom_start=7)

# Додаємо маркери на карту
for index, row in location_fatalities.iterrows():
    # Обчислюємо розмір маркера на основі логарифмічної шкали для зменшення різниці в розмірах
    radius = np.log(row['fatalities'] + 1) * 5
    tooltip_text = f"Location: {row['location']}, Fatalities: {row['fatalities']}"
    folium.CircleMarker(location=[row['latitude'], row['longitude']], radius=radius/1.5, color='red', fill=True, tooltip=tooltip_text).add_to(mymap)

# Відображаємо карту
mymap.save('fatalities_map.html')





In [30]:
import folium
from folium.plugins import TimestampedGeoJson
import pandas as pd
import numpy as np

# Перетворюємо стовпець event_date на тип дати
expl_df['event_date'] = pd.to_datetime(expl_df['event_date'])

# Групуємо дані за місяцем і розташуванням і обчислюємо загальну кількість смертей для кожного розташування
monthly_location_fatalities = expl_df[expl_df['fatalities'] > 0].groupby(['latitude', 'longitude', 'location', pd.Grouper(key='event_date', freq='M')])['fatalities'].sum().reset_index()

# Перетворюємо дані у формат, придатний для TimestampedGeoJson
features = []
for index, row in monthly_location_fatalities.iterrows():
    feature = {
        'type': 'Feature',
        'geometry': {
            'type': 'Point',
            'coordinates': [row['longitude'], row['latitude']],
        },
        'properties': {
            'time': row['event_date'].date().__str__(),
            'popup': f"Location: {row['location']}, Fatalities: {row['fatalities']}",
            'icon': 'circle',
            'iconstyle': {
                'fillColor': 'red',
                'fillOpacity': 0.6,
                'stroke': 'false',
                'radius': np.log(row['fatalities'] + 1) * 3,  # Зменшуємо множник
            }
        }
    }
    features.append(feature)

# Створюємо карту
mymap = folium.Map(location=[monthly_location_fatalities['latitude'].mean(), monthly_location_fatalities['longitude'].mean()], zoom_start=5)

# Додаємо TimestampedGeoJson на карту
TimestampedGeoJson(
    {'type': 'FeatureCollection', 'features': features},
    period='P1M',
    add_last_point=True,
).add_to(mymap)

# Відображаємо карту
mymap.save('monthly_fatalities_animation.html')


In [42]:
# Фільтруємо дані за типами подій
shelling_data = expl_df[expl_df['sub_event_type'] == 'Shelling/artillery/missile attack']
air_strike_data = expl_df[expl_df['sub_event_type'] == 'Air/drone strike']
explosive_data = expl_df[expl_df['sub_event_type'] == 'Remote explosive/landmine/IED']


# Фільтруємо дані за типами подій
shelling_data = expl_df[expl_df['sub_event_type'] == 'Shelling/artillery/missile attack']
air_strike_data = expl_df[expl_df['sub_event_type'] == 'Air/drone strike']
explosive_data = expl_df[expl_df['sub_event_type'] == 'Remote explosive/landmine/IED']

# Групуємо дані за унікальними значеннями location
shelling_locations = shelling_data.groupby(['latitude', 'longitude'])['location'].first().reset_index()
air_strike_locations = air_strike_data.groupby(['latitude', 'longitude'])['location'].first().reset_index()
explosive_locations = explosive_data.groupby(['latitude', 'longitude'])['location'].first().reset_index()

# Створюємо карту з центром у середніх координатах
mymap = folium.Map(location=[expl_df['latitude'].mean(), expl_df['longitude'].mean()], zoom_start=7)

# Додаємо кружечки на карту для кожного типу подій з різними кольорами та ховером
for data, color, event_type in zip([shelling_locations, air_strike_locations, explosive_locations], ['red', 'blue', 'green'], ['Shelling/artillery/missile attack', 'Air/drone strike', 'Remote explosive/landmine/IED']):
    for index, row in data.iterrows():
        folium.CircleMarker(location=[row['latitude'], row['longitude']], radius=5, color=color, fill=True, tooltip=f"Location: {row['location']}\nEvent Type: {event_type}").add_to(mymap)

# Додаємо легенду
legend_html = """
     <div style="position: fixed; bottom: 50px; left: 50px; width: 200px; height: 80px; 
     background-color: white; border:2px solid grey; z-index:9999; font-size:11px;">
     &nbsp; <b>Legend</b> <br>
     &nbsp; Shelling/Artillery/Missile Attack &nbsp; <i class="fa fa-circle fa-1x" style="color:red"></i><br>
     &nbsp; Air/Drone Strike &nbsp; <i class="fa fa-circle fa-1x" style="color:blue"></i><br>
     &nbsp; Remote Explosive/Landmine/IED &nbsp; <i class="fa fa-circle fa-1x" style="color:green"></i><br>
      </div>
     """

mymap.get_root().html.add_child(folium.Element(legend_html))

# Відображаємо карту
mymap.save('events_map.html')
